In [25]:
import pandas as pd
hist = pd.read_csv("/Users/jaeho/Work/yeoboya/recommender/data/tabula/m_yeoboya_ai_dv_mem_item_use_hist.csv")

In [26]:
valid_code = {
    "mem_concn",
    "msg_10day",
    "msg_15day",
    "msg_1day",
    "msg_30day",
    "msg_30day_2",
    "msg_5day",
    "msg_o1hr",
    "pf_view",
    "ps_view_10",
    "ps_view_15",
    "ps_view_30",
    "ps_view_5",
    "vo_mt"
}

In [27]:
from collections import defaultdict

user_rating = defaultdict(dict)

for _, row in hist.iterrows():
    if row['item_code'] in valid_code:
        if row['ptr_mem_no'] in user_rating[row['mem_no']]:
            if user_rating[row['mem_no']][row['ptr_mem_no']] < 10:
                user_rating[row['mem_no']][row['ptr_mem_no']] += 2
        else:
            user_rating[row['mem_no']][row['ptr_mem_no']] = 2
        

In [ ]:
######################### WRITING DATA #########################
import pickle
with open('./user_rating_test_data.pkl', 'wb') as f:
    pickle.dump(user_rating, f)

In [36]:
import pandas as pd
data = pd.read_csv('/Users/jaeho/Work/yeoboya/recommender/data/tabula/member_mate.csv')

/var/folders/23/z66xfw69593_wvynw4k7bb080000gn/T/ipykernel_42827/3952582950.py:2: DtypeWarning: Columns (22,26,33,34,35,36,37,45,46,50,51,59,63,64,65,66,72,80) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/jaeho/Work/yeoboya/recommender/data/tabula/member_mate.csv')


In [37]:
data.shape

(114744, 87)

In [38]:
import pickle
with open('/Users/jaeho/Work/yeoboya/recommender/data/pkl_files/user_rating_test_data.pkl', 'rb') as f:
    user_rating = pickle.load(f)
rated_user = set()
rated_user2 = set()
rated_user3 = set()


for k, v in user_rating.items():
    rated_user.add(k)
    rated_user3.add(k)
    for u,r in v.items():
        rated_user2.add(u)
        rated_user3.add(u)

print(len(rated_user))
print(len(rated_user2))
print(len(rated_user3))

27448
23753
49612


In [39]:
new_user_df = pd.DataFrame()

for idx, row in data.iterrows():
    if row['mem_no'] in rated_user3:
        new_user_df = pd.concat([new_user_df, pd.DataFrame(row).T])

In [40]:
new_user_df.shape

(17920, 87)

In [41]:
mem_id = set(new_user_df['mem_no'].values)
len(mem_id)

17920

In [42]:
import pandas as pd
data = pd.read_csv('/Users/jaeho/Work/yeoboya/recommender/data/tabula/m_yeoboya_ai_dv_member_mate_exit.csv')

/var/folders/23/z66xfw69593_wvynw4k7bb080000gn/T/ipykernel_42827/2266004855.py:2: DtypeWarning: Columns (0,5,14,15,20,21,26,27,28,29,30,31,32,37,38,40,41,43,44,46,47,48,50,51,52,53,54,55,56,58,59,60,61,63,67,75,76) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/jaeho/Work/yeoboya/recommender/data/tabula/m_yeoboya_ai_dv_member_mate_exit.csv')


In [45]:
exit_user_df = pd.DataFrame()

for idx, row in data.iterrows():
    if row['mem_no'] in rated_user3 and row['mem_no'] not in mem_id:
        exit_user_df = pd.concat([exit_user_df, pd.DataFrame(row).T])

In [46]:
exit_user_df.shape

(7129, 83)

In [49]:
new_data_frame = pd.concat([new_user_df, exit_user_df])
print(new_data_frame.shape)
new_data_frame.head()

(25049, 89)


,mem_no,mem_sex,mem_loc,mem_l_code,mem_birth_year,mem_birth_month,mem_birth_day,mem_birth_date,mem_birth_ddi,mate_slct,...,secret_yn,respect_person_cont,my_point_cont,list_prt_yn,my_hd_item_yn,my_hd_yn,ins_date,upd_date,chrgr_name,exit_date
0,5,m,q,qnz,1985,6,3,1985-06-03,5,w,...,n,NaN,90.0,y,n,n,2015-07-22 16:31:35,2023-06-27 14:18:35,NaN,NaN
6,211,m,e,e05,1966,2,0,0000-00-00,10,r,...,n,NaN,100.0,y,n,n,2015-08-09 13:48:39,2023-06-17 22:57:48,NaN,NaN
10,1350974,m,l,l04,1977,10,21,1977-10-21,9,r,...,n,NaN,NaN,y,n,n,2020-10-25 13:25:37,2023-06-28 10:42:26,NaN,NaN
19,120722,m,g,g03,1971,3,0,0000-00-00,3,r,...,n,NaN,88.0,y,n,n,2017-03-12 10:34:49,2023-06-10 10:31:31,NaN,NaN
50,1451758,m,n,n12,1977,12,0,0000-00-00,9,r,...,n,NaN,80.0,y,n,n,2021-01-30 21:38:29,2023-06-19 20:04:55,NaN,NaN


In [61]:
new_data_frame = new_data_frame.drop(columns=["chrgr_name", "exit_date"])

In [51]:
train_mem_id = set(list(map(int, new_data_frame['mem_no'].tolist())))
print(len(train_mem_id))

25049


In [52]:
import pickle
with open('/Users/jaeho/Work/yeoboya/recommender/data/pkl_files/user_rating_test_data.pkl', 'rb') as f:
    user_rating_data = pickle.load(f)

In [53]:
print(len(user_rating_data))

27448


In [54]:
from collections import defaultdict
filtered_user_rating = defaultdict(dict)

for k, v in user_rating_data.items():
    if k in train_mem_id:
        for u, r in v.items():
            if u in train_mem_id:
                filtered_user_rating[k][u] = r

print(len(filtered_user_rating))

11892


In [55]:
######################### WRITING DATA #########################
import pickle
with open("label_user_rating.pkl", "wb") as f:
    pickle.dump(filtered_user_rating, f)

In [65]:
import pickle
import pandas as pd

with open("/Users/jaeho/Work/yeoboya/recommender/data/nn_data/label/label_user_rating.pkl", "rb") as f:
    user_rating = pickle.load(f)
user_info = pd.read_csv("/Users/jaeho/Work/yeoboya/recommender/data/nn_data/train/train_user_data.csv")

/var/folders/23/z66xfw69593_wvynw4k7bb080000gn/T/ipykernel_42827/1504141475.py:6: DtypeWarning: Columns (7,22,25,26,33,34,37,63,64,65,66,82) have mixed types. Specify dtype option on import or set low_memory=False.
  user_info = pd.read_csv("/Users/jaeho/Work/yeoboya/recommender/data/nn_data/train/train_user_data.csv")


In [66]:
print(len(user_rating))
print(user_info.shape)
user_info.head()

11892
(20696, 87)


,mem_no,mem_sex,mem_loc,mem_l_code,mem_birth_year,mem_birth_month,mem_birth_day,mem_birth_date,mem_birth_ddi,mate_slct,...,family_conts_upd_cnt,secret_item_yn,secret_yn,respect_person_cont,my_point_cont,list_prt_yn,my_hd_item_yn,my_hd_yn,ins_date,upd_date
0,5,m,q,qnz,1985.0,6.0,3.0,1985-06-03,5.0,w,...,14.0,n,n,NaN,90.0,y,n,n,2015-07-22 16:31:35,2023-06-27 14:18:35
1,211,m,e,e05,1966.0,2.0,0.0,0000-00-00,10.0,r,...,9.0,n,n,NaN,100.0,y,n,n,2015-08-09 13:48:39,2023-06-17 22:57:48
2,1350974,m,l,l04,1977.0,10.0,21.0,1977-10-21,9.0,r,...,102.0,n,n,NaN,NaN,y,n,n,2020-10-25 13:25:37,2023-06-28 10:42:26
3,120722,m,g,g03,1971.0,3.0,0.0,0000-00-00,3.0,r,...,5.0,n,n,NaN,88.0,y,n,n,2017-03-12 10:34:49,2023-06-10 10:31:31
4,1845,m,b,b16,1965.0,10.0,0.0,0000-00-00,9.0,r,...,1.0,n,n,NaN,80.0,y,n,n,2015-09-09 23:16:54,2022-11-02 21:55:29


In [67]:
print(len(user_rating))

11892


In [68]:
user_id_dict = {}
used_id_set = set()

id = 0
for user_id, target in user_rating.items():
    if user_id not in user_id_dict:
        user_id_dict[user_id] = id
        used_id_set.add(id)
        id += 1
        
    for ptr_id, rating in target.items():
        if ptr_id not in user_id_dict:
            user_id_dict[ptr_id] = id
            used_id_set.add(id)
            id += 1

print(len(used_id_set))
print(len(user_id_dict))

20696
20696


In [72]:
from collections import defaultdict

new_rating = defaultdict(dict)

for user_id, target in user_rating.items():
    for ptr_id, rating in target.items():
        new_rating[user_id_dict[user_id]][user_id_dict[ptr_id]] = rating
print(len(new_rating))

11892


In [75]:
######################### WRITING DATA #########################
import pickle

with open("new_user_rating.pkl", "wb") as f:
    pickle.dump(new_rating, f)

In [77]:
import pandas as pd
user_info = pd.read_csv("/Users/jaeho/Work/yeoboya/recommender/data/nn_data/train/train_user_data.csv")
user_info.head()

/var/folders/23/z66xfw69593_wvynw4k7bb080000gn/T/ipykernel_42827/712380287.py:2: DtypeWarning: Columns (7,22,25,26,33,34,37,63,64,65,66,82) have mixed types. Specify dtype option on import or set low_memory=False.
  user_info = pd.read_csv("/Users/jaeho/Work/yeoboya/recommender/data/nn_data/train/train_user_data.csv")


,mem_no,mem_sex,mem_loc,mem_l_code,mem_birth_year,mem_birth_month,mem_birth_day,mem_birth_date,mem_birth_ddi,mate_slct,...,family_conts_upd_cnt,secret_item_yn,secret_yn,respect_person_cont,my_point_cont,list_prt_yn,my_hd_item_yn,my_hd_yn,ins_date,upd_date
0,5,m,q,qnz,1985.0,6.0,3.0,1985-06-03,5.0,w,...,14.0,n,n,NaN,90.0,y,n,n,2015-07-22 16:31:35,2023-06-27 14:18:35
1,211,m,e,e05,1966.0,2.0,0.0,0000-00-00,10.0,r,...,9.0,n,n,NaN,100.0,y,n,n,2015-08-09 13:48:39,2023-06-17 22:57:48
2,1350974,m,l,l04,1977.0,10.0,21.0,1977-10-21,9.0,r,...,102.0,n,n,NaN,NaN,y,n,n,2020-10-25 13:25:37,2023-06-28 10:42:26
3,120722,m,g,g03,1971.0,3.0,0.0,0000-00-00,3.0,r,...,5.0,n,n,NaN,88.0,y,n,n,2017-03-12 10:34:49,2023-06-10 10:31:31
4,1845,m,b,b16,1965.0,10.0,0.0,0000-00-00,9.0,r,...,1.0,n,n,NaN,80.0,y,n,n,2015-09-09 23:16:54,2022-11-02 21:55:29


In [80]:
for i, row in user_info.iterrows():
    user_info.loc[i, "mem_no"] = user_id_dict[row["mem_no"]]
user_info.head()

,mem_no,mem_sex,mem_loc,mem_l_code,mem_birth_year,mem_birth_month,mem_birth_day,mem_birth_date,mem_birth_ddi,mate_slct,...,family_conts_upd_cnt,secret_item_yn,secret_yn,respect_person_cont,my_point_cont,list_prt_yn,my_hd_item_yn,my_hd_yn,ins_date,upd_date
0,19083,m,q,qnz,1985.0,6.0,3.0,1985-06-03,5.0,w,...,14.0,n,n,NaN,90.0,y,n,n,2015-07-22 16:31:35,2023-06-27 14:18:35
1,12415,m,e,e05,1966.0,2.0,0.0,0000-00-00,10.0,r,...,9.0,n,n,NaN,100.0,y,n,n,2015-08-09 13:48:39,2023-06-17 22:57:48
2,11191,m,l,l04,1977.0,10.0,21.0,1977-10-21,9.0,r,...,102.0,n,n,NaN,NaN,y,n,n,2020-10-25 13:25:37,2023-06-28 10:42:26
3,8558,m,g,g03,1971.0,3.0,0.0,0000-00-00,3.0,r,...,5.0,n,n,NaN,88.0,y,n,n,2017-03-12 10:34:49,2023-06-10 10:31:31
4,20641,m,b,b16,1965.0,10.0,0.0,0000-00-00,9.0,r,...,1.0,n,n,NaN,80.0,y,n,n,2015-09-09 23:16:54,2022-11-02 21:55:29


In [81]:
######################### WRITING DATA #########################
user_info.to_csv("new_user_data.csv", index=False)

In [82]:
user_info.shape

(20696, 87)